In [ ]:
# from deap import base, creator, tools, algorithms
from dedalus import public as de
from dedalus.extras.plot_tools import plot_bot_2d
from dedalus.extras import *
import numpy as np
import scipy
from scipy import integrate
from scipy.stats import norm
import pickle
from math import sqrt
import matplotlib.pyplot as plt
import pandas as pd


import time as timeski
import os

#Suppress most Dedalus output
de.logging_setup.rootlogger.setLevel('ERROR')

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'arial'
# rcParams['font.sans-serif'] = ['Tahoma']
rcParams.update({'font.size': 16})

In [ ]:
###################################################################
# Simulation Functions
###################################################################

#Scale list between 0 and 1
def scale(A):
    return (A-np.min(A))/(np.max(A) - np.min(A))

# steady state ODE
def ssODE(y,t,params):
    u,v,u2,v2,g,G = y
    b,gamma,n,sigma,e,c,d,k,delta,K = params
    
    du = b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u
    dv = -du
    
    du2 = k*v2-delta*u2
    dv2 = -du2
    
    dG = c*u*g-d*G
    dg = - dG
    
    derivs = [du,dv,du2,dv2,dg,dG]
    return derivs

#determine homogenous SS using ssODE
def homogenousSS(u,v,u2,v2,g,G,params):

    y0 = (u,v,u2,v2,g,G)
    t_sim = (0,1000)
    odeSoln = integrate.odeint(ssODE,y0,t_sim,args=(params,),mxstep=1000000) 
      
    return(odeSoln[1])


def TwoStep_WPGAP_2D_GaussRing(A_b = 0,A_amp=1,var = 0.2,Du = 0.04, DG_factor = 100,
                     feedback = 'none'):

    #Bases:names,modes,intervals,dealiasing
    phi_basis=de.Fourier('p',256,interval=(0,2*np.pi),dealias=3/2)
    r_basis=de.Chebyshev('r',128,interval=(0,4),dealias=3/2)
    #Domain:bases,datatype
    domain=de.Domain([phi_basis,r_basis],float)
    phi, r = domain.grids(scales=1)
    

    mu_F = 1.75
    
    As = domain.new_field(name='As')

    norm_dist = norm.pdf(r[0],2,var)
    norm_dist_scaled = scale(norm_dist)
    As['g'] = np.array(A_b+A_amp*norm_dist_scaled)

    As.meta['p']['constant'] = True
    
    #if using scale factor to scale GTPase concentration
    scale_f = 200

    T_mod = 2
    T = 4.04*scale_f
    Tg = 10
    b = 1
    c = 10/scale_f
    sigma = 10
    e = 10
    gamma = 20
    n = 2
    d = 10
    K = 1*scale_f

#     Du = .004
    Dv = 100*Du
    DG = DG_factor*Du #40*Du
    Dg = 100*Du

#   b,gamma,n,sigma,e,c,d,k,delta = params
    p_ss = [b,gamma,n,sigma,e,c,d,0,0,K]
    u0,v0,u20,v20,g0,G0 = homogenousSS(T/2,T/2,T_mod/2,T_mod/2, Tg/2, Tg/2,p_ss)



    # Specify problem
    problem = de.IVP(domain, variables=['u', 'v','ur','vr','G','g','Gr','gr'])
    
    problem.parameters['u0'] = u0
    problem.parameters['v0'] = v0

    problem.parameters['G0'] = G0
    problem.parameters['g0'] = g0
    problem.parameters['As'] = As
    problem.parameters['Tg'] = Tg
    
    problem.parameters['gamma'] = gamma
    problem.parameters['b'] = b
    problem.parameters['n'] = n
    problem.parameters['c'] = c
    problem.parameters['dd'] = d
    problem.parameters['sigma'] = sigma
    problem.parameters['e'] = e
    problem.parameters['K'] = K

    
    problem.parameters['Du'] = Du
    problem.parameters['Dv'] = Dv

    problem.parameters['DG'] = DG
    problem.parameters['Dg'] = Dg
    

    
    if feedback == 'none':
        problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u'
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-dd*G'
        
#     ### require ring of modulator

        
#GTPase activation
    if feedback == 'b':
        problem.parameters['A_max'] = np.max(As)
        problem.substitutions['f(u,v,G)'] = '(As)*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u'
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-dd*G'
        
    #GTPase self-positive feedback
    if feedback == 'gamma':
#         problem.parameters['A_max'] = np.max(As)
        problem.substitutions['f(u,v,G)'] = 'b*v+(As)*v*u**n/(K**n+u**n)-sigma*u-e*G*u'
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-dd*G'
        
    #GAP inactivation
    if feedback == 'dd':
        problem.parameters['A_max'] = np.max(As)
        problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u'
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-(As)*G'
        
            #GAP inactivation
    if feedback == 'dd_noscale':
        problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u'
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-(As)*G'
        
    ### require inverted ring of modulator
    
    #GTPase inactivation
    if feedback == 'sigma':
#         problem.parameters['A_max'] = np.max(As)
#         problem.parameters['A_max'] = np.min(As)*feed_mod
        problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-(As)*u-e*G*u' 
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-dd*G'
        
    #GAP-mediated GTPase inactivation
    if feedback == 'e':
#         problem.parameters['A_max'] = np.min(As)*feed_mod
        problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-(As)*G*u' 
        problem.substitutions['fg(u,G,g)'] = 'c*u*g-dd*G'
        
    #GAP activation   
    if feedback == 'c':
#         problem.parameters['A_max'] = np.min(As)*feed_mod
        problem.parameters['A_max'] = np.max(As)
        problem.substitutions['f(u,v,G)'] = 'b*v+v*gamma*u**n/(K**n+u**n)-sigma*u-e*G*u'
        problem.substitutions['fg(u,G,g)'] = '(As)*u*g-dd*G' 

    # Mass always conserved
    problem.substitutions['minf(u,v,G)'] = '-f(u,v,G)'
    problem.substitutions['minfg(u,G,g)'] = '-fg(u,G,g)'


    problem.add_equation("r**2*dt(u)-r**2*Du*dr(ur)-r*Du*dr(u)-Du*dp(dp(u))=r**2*f(u,v,G)")
    problem.add_equation("r**2*dt(v)-r**2*Dv*dr(vr)-r*Dv*dr(v)-Dv*dp(dp(v))=r**2*minf(u,v,G)")
    problem.add_equation("r**2*dt(G)-r**2*DG*dr(Gr)-r*DG*dr(G)-DG*dp(dp(G))=r**2*fg(u,G,g)")
    problem.add_equation("r**2*dt(g)-r**2*Dg*dr(gr)-r*Dg*dr(g)-Dg*dp(dp(g))=r**2*minfg(u,G,g)")


    problem.add_equation("ur-dr(u)=0")
    problem.add_equation("vr-dr(v)=0")

    problem.add_equation("Gr-dr(G)=0")
    problem.add_equation("gr-dr(g)=0")

    #Reflective boundary conditions

    problem.add_bc("left (ur) = 0")
    problem.add_bc("right (ur) = 0")
    problem.add_bc("left (vr) = 0")
    problem.add_bc("right (vr) = 0")
    problem.add_bc("left (Gr) = 0")
    problem.add_bc("right (Gr) = 0")
    problem.add_bc("left (gr) = 0")
    problem.add_bc("right (gr) = 0")


    # Pick a timestepper
    ts = de.timesteppers.RK443 #MCNAB2
    # Build solver
    solver = problem.build_solver(ts)
    # Set integration limits
    solver.stop_wall_time = np.inf
    solver.stop_sim_time = np.inf
    solver.stop_iteration = np.inf
    # Set initial conditions
    u = solver.state ['u']
    v = solver.state['v']
    G = solver.state ['G']
    g = solver.state['g']
#     u_seed = pickle.load( open( "PR_9Dots.pickle", "rb" ) )
#     u_seed_norm = u_seed/np.max(u_seed)
#     urand = 0.05*v0*np.random.rand(*u['g'].shape) + 0.05*v0*u_seed_norm

    urand = 0.3*v0*np.random.rand(*u['g'].shape)
    
#     plt.plot(u0+urand)

    u['g'] = u0+urand
    v['g'] = v0-urand
    G['g'] = G0*np.ones(G['g'].shape)
    g['g'] = g0*np.ones(g['g'].shape)
    
# #  ################################ For movies
#     file_name = 'Figures/TwoStep_WPGAP_dd_bigger_disc_wider_ring_larger_domain'
#     dpi = 300
    
#     phi, r = domain.grids(scales=1)
#     phi = np.vstack((phi,2*np.pi))
#     phi,r = np.meshgrid(phi,r)
#     z = np.vstack((u['g'],u['g'][0])).T
#     fig = plt.figure()
#     # ax = Axes3D(fig)

#     plt.subplot(projection="polar")

#     plt.pcolormesh(phi,r, z,vmin=0., vmax = 1, shading='auto')

#     plt.plot(phi, r, color='k', ls='none') 
#     plt.title('t={}'.format(0))
#     plt.xticks([])
#     plt.yticks([])
# #     plt.colorbar()
    
#     plt.savefig(file_name+'_00{}.png'.format(0),dpi=dpi)
#     plt.close()
# #  ################################ 

    phi, r = domain.grids(scales=domain.dealias)
    phi = np.vstack((phi,2*np.pi))
    phi,r = np.meshgrid(phi,r)


    solver.stop_iteration = 40000
    
    dt = 0.025 
    nonan=True
    not_steady = True
    prev_state = np.zeros((256*3//2,128*3//2))
    # Main loop chceking stopping criteria
    while solver.ok and nonan and not_steady:
        # Step forward
        solver.step(dt)

        if solver.iteration % 50 == 0:
            if np.count_nonzero(np.isnan(u['g'])) > 0 or np.min(u['g']) < 0 :
                return('Numerical Error')
                nonan = False  
                
        if solver.iteration% 50 ==0:
            curr_state = np.array(u['g'])
            if np.max(np.abs(curr_state-prev_state)) < 10e-2:
                print(np.max(np.abs(curr_state-prev_state)))
                print('Steady state at t = %.2f'%(np.round(solver.iteration*dt,2)))
                not_steady = False
            else: prev_state = np.array(u['g'])
       
    #Plot active GTPase
    z = np.vstack((u['g'],u['g'][0])).T
    fig = plt.figure(figsize=(4,4))
    plt.subplot(projection="polar")

    plt.pcolormesh(phi,r,z,shading='auto',vmin= 0.2*scale_f, vmax = 0.8*scale_f)

    plt.plot(phi, r, color='k', ls='none') 
    plt.xticks([])
    plt.yticks([])
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('[u]',rotation=0)
#     plt.savefig('TwoStepFigures/TwoStep_u_Feedback_%s.pdf'%(feedback),bbox_inches='tight',dpi=300)
#     plt.savefig('TwoStepFigures/TwoStep_u_Feedback_%s.png'%(feedback),bbox_inches='tight',dpi=300)
#     plt.close()
    
#     #Plot other species
#     species = ['v','G*','g']
#     for i,spec in enumerate([v['g'],G['g'],g['g']]):
#         z = np.vstack((spec,spec[0])).T

#         fig = plt.figure(figsize=(4,4))
#         plt.subplot(projection="polar")

#         plt.pcolormesh(phi,r, z, shading='auto')
#         plt.plot(phi, r, color='k', ls='none') 
        
#         plt.xticks([])
#         plt.yticks([])
        
#         cbar = plt.colorbar(fraction=0.046, pad=0.04)
#         cbar.ax.get_yaxis().labelpad = 15
#         cbar.ax.set_ylabel('[%s]'%species[i],rotation=0)
        
# #         plt.savefig('TwoStepFigures/TwoStep_%s_Feedback_%s.pdf'%(species[i],feedback),bbox_inches='tight',dpi=300)
# #         plt.savefig('TwoStepFigures/TwoStep_%s_Feedback_%s.png'%(species[i],feedback),bbox_inches='tight',dpi=300)
# #         plt.close()

#     #Plot modulator species
#     z = np.vstack((As['g'],As['g'][0])).T

#     fig = plt.figure(figsize=(4,4))
#     plt.subplot(projection="polar")
#     plt.pcolormesh(phi,r, z, shading='auto')#,vmin=0., vmax = 0.8, shading='auto')
#     plt.plot(phi, r, color='k', ls='none') 
#     plt.xticks([])
#     plt.yticks([])

#     cbar = plt.colorbar(fraction=0.046, pad=0.04)
#     cbar.ax.get_yaxis().labelpad = 15
#     cbar.ax.set_ylabel('[M]',rotation=0)
# # #     plt.savefig('TwoStepFigures/TwoStep_M_Feedback_%s.png'%(feedback),bbox_inches='tight',dpi=300)
# # #     plt.savefig('TwoStepFigures/TwoStep_M_Feedback_%s.pdf'%(feedback),bbox_inches='tight',dpi=300)

    return u['g'].T


### Linear basal + modulated scenarios

In [ ]:
rs = np.linspace(0,4,1000)

In [ ]:
plt.figure(figsize=(4,2))

A_b = 0.5
A_amp=0.05

norm_pdf = norm.pdf(rs,2,0.2)

gamma = np.array(A_b+A_amp*scale(norm_pdf))

plt.plot(rs,gamma)
plt.ylabel('[M]')
plt.xlabel('Radius ($\mu m$)')
plt.title('M modulates $b$')

# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.pdf'%('b'),bbox_inches='tight',dpi=300)
# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.png'%('b'),bbox_inches='tight',dpi=300)

In [ ]:
# u = TwoStep_WPGAP_2D_GaussRing(A_b = 0.04,A_amp=0.005,var = 0.2,feedback='b')
u = TwoStep_WPGAP_2D_GaussRing(A_b = 0.5,A_amp=0.05,var = 0.2,feedback='b')


In [ ]:
plt.figure(figsize=(4,2))

A_b = 15
A_amp=2.5

norm_pdf = norm.pdf(rs,2,0.2)

gamma = np.array(A_b+A_amp*scale(norm_pdf))

plt.plot(rs,gamma)
plt.ylabel('[M]')
plt.xlabel('Radius ($\mu m$)')
plt.title('M modulates $\gamma$')

# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.pdf'%('gamma'),bbox_inches='tight',dpi=300)
# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.png'%('gamma'),bbox_inches='tight',dpi=300)

In [ ]:
#Self-positive feedback rate
u = TwoStep_WPGAP_2D_GaussRing(A_b = 15,A_amp=2.5,Du = 0.04,feedback='gamma')



In [ ]:
plt.figure(figsize=(4,2))

A_b = 17.5
A_amp=-2.5

norm_pdf = norm.pdf(rs,2,0.2)

gamma = np.array(A_b+A_amp*scale(norm_pdf))

feedback = 'sigma'

plt.plot(rs,gamma)
plt.ylabel('[M]')
plt.xlabel('Radius ($\mu m$)')
plt.title('M modulates $\sigma$')


# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.pdf'%(feedback),bbox_inches='tight',dpi=300)
# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.png'%(feedback),bbox_inches='tight',dpi=300)

In [ ]:
#GTPase Inactivation Rate
u = TwoStep_WPGAP_2D_GaussRing(A_b = 17.5,A_amp=-2.5,feedback='sigma') 


In [ ]:
plt.figure(figsize=(4,2))

A_b = 18.5
A_amp=-4.5

norm_pdf = norm.pdf(rs,2,0.2)

gamma = np.array(A_b+A_amp*scale(norm_pdf))

feedback = 'e'

plt.plot(rs,gamma)
plt.ylabel('[M]')
plt.xlabel('Radius ($\mu m$)')
plt.title('M modulates $e$')

# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.pdf'%(feedback),bbox_inches='tight',dpi=300)
# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.png'%(feedback),bbox_inches='tight',dpi=300)


In [ ]:
#GAP feedback
u = TwoStep_WPGAP_2D_GaussRing(A_b = 18.5,A_amp=-4.5,feedback='e')


GAP kinetics are a bit more fickle

In [ ]:
plt.figure(figsize=(4,2))

scale_f = 200

A_b = 17/scale_f
A_amp=-7.2/scale_f
var = 0.3

norm_pdf = norm.pdf(rs,2,var)

cs = np.array(A_b+A_amp*scale(norm_pdf))

feedback = 'c'

plt.plot(rs,cs)
plt.ylabel('[M]')
plt.xlabel('Radius ($\mu m$)')
plt.title('M modulates $c$')

# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.pdf'%(feedback),bbox_inches='tight',dpi=300)
# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.png'%(feedback),bbox_inches='tight',dpi=300)


In [ ]:

u = TwoStep_WPGAP_2D_GaussRing(A_b = 17/200,A_amp=-7.2/200,var = 0.3,feedback='c')#,n_iter=3000)


In [ ]:
plt.figure(figsize=(4,2))

A_b = 4
A_amp=7.5
var = 0.3

norm_pdf = norm.pdf(rs,2,var)

gamma = np.array(A_b+A_amp*scale(norm_pdf))

feedback = 'd'

plt.plot(rs,gamma)
plt.ylabel('[M]')
plt.xlabel('Radius ($\mu m$)')
plt.title('M modulates $d$')

# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.pdf'%(feedback),bbox_inches='tight',dpi=300)
# plt.savefig('TwoStepFigures/TwoStep_M_1D_Feedback_%s.png'%(feedback),bbox_inches='tight',dpi=300)


In [ ]:
u = TwoStep_WPGAP_2D_GaussRing(A_b = 4,A_amp=7.5,var = 0.3,feedback='dd')#,n_iter=1000)
